In [1]:
import zmq
import threading
import tkinter as tk
from ocpp.v16 import ChargePoint as cp
from ocpp.v16 import call_result

In [2]:
class ChargePoint(cp):
    async def boot_notification(self, charge_point_vendor, charge_point_model, **kwargs):
        print("Boot notification received")
        return call_result.BootNotificationPayload(
            current_time="2024-04-22T12:34:56Z",
            interval=300,
            status="Accepted"
        )

    async def heartbeat(self):
        print("Heartbeat received")
        return call_result.HeartbeatPayload(
            current_time="2023-04-22T12:34:56Z"
        )

In [3]:
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5557")

<SocketContext(bind='tcp://*:5557')>

In [4]:
root = tk.Tk()
root.title("Charging Station")

status_label = tk.Label(root, text="Station Status: Idle")
status_label.pack()

In [5]:
def worker():
    while True:
        message = socket.recv()
        print(f"Received request: {message}")

        # Process the request and send a response
        if message == b"start_charging":
            response = b"Charging started"
            status_label.config(text="Station Status: Charging")
        elif message == b"stop_charging":
            response = b"Charging stopped"
            status_label.config(text="Station Status: Idle")
        else:
            response = b"Invalid request"

        socket.send(response)

In [ ]:
def main():
    worker_thread = threading.Thread(target=worker)
    worker_thread.start()

    root.mainloop()

if __name__ == "__main__":
    main()

Received request: b'start_charging'
Received request: b'stop_charging'
Received request: b'start_charging'
Received request: b'stop_charging'
